# AnoGANの実装

In [1]:
import os 
import sys
import shutil
import pickle
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader
from torchvision import transforms
import gc

In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim=20, image_size=64):
        super(Generator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(z_dim, image_size*8,kernel_size=4,stride=1),
            nn.BatchNorm2d(image_size * 8),
            nn.Relu(inplace=True))
        
        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(image_size*8, image_size*4, 
                               kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(image_size * 4),
            nn.Relu(implace=True))
        
        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(image_size*4, image_size*2, 
                               kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(image_size * 2),
            nn.Relu(implace=True))
        
        self.layer4 = nn.Sequential(
            nn.ConvTranspose2d(image_size*2, image_size, 
                               kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(image_size),
            nn.Relu(implace=True))
        
        self.last = nn.Sequential(
            nn.ConvTranspose2d(image_size,1,kernel_size=4,stride=2,padding=1),
            nn.Tahn())
        
    def forward(self, z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.laeyer4(out)
        out = self.last(out)
        
        return out

In [5]:
class Discriminator(nn.Module):
    def __init__(self, z_dim=20, image_size=64):
        super(Dicriminator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, image_size, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(image_size, image_size*2, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(image_size*2, image_size*4, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(image_size*4, image_size*8, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))
        
        self.last = nn.Conv2d(image_size*8,1, kernelsize=4, stride=1)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        feature = out
        feature = feature.view(feature.size()[0], -1)
        
        out = self.last(out)
        return out , feature

In [10]:
z = torch.randn(5,20)
z = z.view(z.size(0), z.size(1), 1, 1)
z.shape

torch.Size([5, 20, 1, 1])

In [7]:
z.requires_grad = True
z_optimizer = torch.optim.Adam([z], lr = 1e-3)

## 損失関数の定義

In [11]:
def Anomaly_score(x, fake_img, D, Lambda=0.1):
    #Pixel単位で並列化処理して絶対値を合算
    residual_loss = torch.abs(x - fake_img)
    residual_loss = residual_loss.view(residual_loss.size()[0], -1)
    residual_loss = torch.sum(residual_loss, dim=1)
    
    _, x_feature = D(x)
    _, G_feature = D(fake_img)
    
    #Discriminatorの一つ前のLayerの特徴量のfakeとrealの画像の差額の絶対値を合算
    discriminator_loss = torch.abs(x_feature - G_feature)
    discriminator_loss = discriminator_loss.view(discriminator_loss.size(), -1)
    discriminator_loss = discriminator_loss.sum(discriminator_loss,dim=1)
    
    loss_each = (1-Lambda)*residual_loss + Lambda*discriminator_loss
    total_loss = torch.sum(loss_each)
    
    return total_loss, loss_each, residual_loss

In [ ]:
EPOCHS = 5000

In [ ]:
x = image[0:5]
x = x.to(device)

z = torch.randn(5,20)
z = z.view(z.size(0), z.size(1), 1, 1)
z.requires_grad = True
z_optimizer = torch.optim.Adam([z], lr = 1e-3)

for epoch in range(EPOCHS):
    fake_image = G_update(z)
    loss, _, _ = Anomaly_score(x, fake_img, D_update, Lambda=0.1)
    
    z_optimizer.zero_grad()
    loss.backward()
    z_optimizer.step()
    
    if epoch % 1000 == 0:
        